In [56]:
import requests
import time
from bs4 import BeautifulSoup
import re
import pandas as pd

def parse_text(unparsed):
    parsed = unparsed.get_text().lstrip('\n\r\n').lstrip('\n\r\n ').rstrip('\n\r\n').rstrip('\n\r\n ')
    return parsed

def extract_uuid(string):
    uuid_regex = re.compile("[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}")
    return re.search(uuid_regex, string).group(0)
    
def remove_non_numeric(string):
    try: 
        string = string.strip().replace(",", "")
        return re.sub("[^0-9^.]", "", string)
    except AttributeError:
        pass

def extract_autotrader_year(string):
    try:
        return remove_non_numeric(string)[:4]
    except TypeError:
        pass

def extract_enginge_size(list):
    engine_size_regex = re.compile("[1-2]{1}[.]{1}[0-8]{1}[L]{1}|[1-2]{1}[.]{1}[0-8]{1}")
    try:
        return re.search(engine_size_regex, list).group()
    except AttributeError:
        pass


#URL Headers
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

In [2]:
#Listings Dicts

cazoo_listings = []

autotrader_listings = []

In [57]:
#Cazoo Listings

cazoo_listings.clear() # delete all values in the cazoo listings dict

cazoo_page = requests.get("https://www.cazoo.co.uk/cars/mercedes-benz/a-class/?colour=Grey&fuelType=Petrol", headers=headers)

cazoo_soup = BeautifulSoup(cazoo_page.content, 'html.parser')
cazoo_search_results = cazoo_soup.find_all("div", {"class" : re.compile('.*vehicle-cardstyles__Card-sc.*')})

for s in cazoo_search_results:
    
    car_id = extract_uuid(s.find('a', href=True)['href'])

    details = s.findAll("div", {"class" : re.compile('.*vehicle-cardstyles__DetailWrap-sc.*')})
    
    for d in details:
        tags = d.find_next('ul').find_all('li')

        title = parse_text(d.find("p", {"class": re.compile('.*vehicle-cardstyles__Title-sc.*')}))
        description = parse_text(d.find("p", {"class" : re.compile('.*vehicle-cardstyles__DisplayVariant-sc.*')}))
        mileage = remove_non_numeric(parse_text(tags[0]))
        year = remove_non_numeric(parse_text(tags[1]))
        transmission = parse_text(tags[2])
        fuel = parse_text(tags[3])
        engine = remove_non_numeric(extract_enginge_size(description))

    price_gbp = remove_non_numeric(parse_text(s.find("p", {"data-test-id": "card-pricing-full-price-gb"})))

    listing_details = { 'car_id': car_id,
                        'car_title': title,
                        'car_decsription': description,
                        'engine': engine,
                        'mileage': mileage,
                        'year': year,
                        'transmission': transmission,
                        'fuel': fuel,
                        'price_gbp': price_gbp,
                        'source': 'Cazoo',
                        'url': f"https://www.cazoo.co.uk/car-details/{car_id}/"
                        }
    
    cazoo_listings.append(listing_details)

print(f"found {len(cazoo_listings)} car listings on Cazoo")


found 22 car listings on Cazoo


In [60]:
#Autotrader Listings

autotrader_listings.clear() # delete all values in the cazoo listings dict

for page in range(1, 40): #checks the first 40 pages of paginated results

    autotrader_page = requests.get(f"https://www.autotrader.co.uk/car-search?sort=price-asc&postcode=WC2N5DU&radius=1500&make=Mercedes-Benz&model=A%20Class&include-delivery-option=on&year-from=2020&fuel-type=Petrol&exclude-writeoff-categories=on&page={page}", headers=headers)
    time.sleep(2) # space requests 2 seconds apart to avoid being rate limited
    autotrader_soup = BeautifulSoup(autotrader_page.content, 'html.parser')
    autotrader_search_results = autotrader_soup.find_all("li", {"class" : "search-page__result"})

    for r in autotrader_search_results: 

        car_id = r.get('id')
        title = parse_text(r.find("h3", {"class": "product-card-details__title"}))
        description = parse_text(r.find("p", {"class": "product-card-details__subtitle"}))
        price_gbp = remove_non_numeric(parse_text(r.find("div", {"class": "product-card-pricing__price"})))

        key_specs = r.find_all("li", {"class": "atc-type-picanto--medium"})
        key_specs_list = [parse_text(key_spec) for key_spec in key_specs]
        
        mileage = remove_non_numeric(next((x for x in key_specs_list if 'miles' in x), None))
        year = extract_autotrader_year(next((x for x in key_specs_list if 'reg)' in x), None))
        fuel = ('Petrol' if 'Petrol' in key_specs_list else 'Diesel')
        transmission = ('Manual' if 'Manual' in key_specs_list else 'Automatic')
        enginge = remove_non_numeric(next((x for x in key_specs_list if extract_enginge_size(x)), None))


        listing_details = { 'car_id': car_id,
                            'car_title': title,
                            'car_decsription': description,
                            'engine': enginge,
                            'mileage': mileage,
                            'year': year,
                            'transmission': transmission,
                            'fuel': fuel,
                            'price_gbp': price_gbp,
                            'source': 'Autotrader',
                            'url': f"https://www.autotrader.co.uk/car-details/{car_id}"
                            }

        autotrader_listings.append(listing_details)

print(f"found {len(autotrader_listings)} car listings on Autotrader")

found 429 car listings on Autotrader


In [61]:
#Combine Listings to DF
print("Combining listings dicts")
listings = cazoo_listings + autotrader_listings
print("Generating df")
df = pd.DataFrame.from_dict(listings)
print(f"Raw df contains {df.shape[0]} records")
print(f"Dropping NAN's")
df = df.dropna()
print(f"Formatted df contains {df.shape[0]} records")

Combining listings dicts
Generating df
Raw df contains 451 records
Dropping NAN's
Formatted df contains 329 records


In [62]:
df.head(40)

,car_id,car_title,car_decsription,engine,mileage,year,transmission,fuel,price_gbp,source,url
0,6a6c65ae-5dff-54dc-a011-14e74ef1a9e8,Mercedes-Benz A Class,1.3L SE A180,1.3,23532,2019,Manual,Petrol,18450,Cazoo,https://www.cazoo.co.uk/car-details/6a6c65ae-5...
1,3d0f6b3d-7703-5310-9a72-e23e3f6fc5b7,Mercedes-Benz A Class,1.3L SE A180,1.3,22971,2019,Manual,Petrol,18900,Cazoo,https://www.cazoo.co.uk/car-details/3d0f6b3d-7...
2,421a7618-3a4f-5c7c-932d-55f1cc981990,Mercedes-Benz A Class,1.3L SE A180,1.3,26920,2020,Manual,Petrol,18925,Cazoo,https://www.cazoo.co.uk/car-details/421a7618-3...
3,202c6763-4214-5129-8567-96cd4c14846a,Mercedes-Benz A Class,1.3L SE A180,1.3,8127,2019,Manual,Petrol,20150,Cazoo,https://www.cazoo.co.uk/car-details/202c6763-4...
4,581282c8-fa6b-53ff-840b-867fc68e3a08,Mercedes-Benz A Class,1.6L AMG Line A200,1.6,35792,2018,Automatic,Petrol,18450,Cazoo,https://www.cazoo.co.uk/car-details/581282c8-f...
5,557a7e76-d915-5482-b56f-506edf5dd342,Mercedes-Benz A Class,1.3L AMG Line A180,1.3,56581,2019,Manual,Petrol,19400,Cazoo,https://www.cazoo.co.uk/car-details/557a7e76-d...
6,d4427748-b030-574c-b0f8-2d70a37e97de,Mercedes-Benz A Class,1.3L Sport A180,1.3,23674,2019,Manual,Petrol,20925,Cazoo,https://www.cazoo.co.uk/car-details/d4427748-b...
7,19f6a21b-edf7-5ef5-8824-aed3d68e5417,Mercedes-Benz A Class,1.3L AMG Line A180,1.3,11879,2019,Manual,Petrol,21325,Cazoo,https://www.cazoo.co.uk/car-details/19f6a21b-e...
8,80ad0c59-cadf-5bb7-9416-af11f861473f,Mercedes-Benz A Class,1.3L AMG Line A200,1.3,31815,2018,Automatic,Petrol,21175,Cazoo,https://www.cazoo.co.uk/car-details/80ad0c59-c...
9,5a77e9fb-96ea-5cb4-880a-7c252bcdf9db,Mercedes-Benz A Class,1.3L AMG Line A200,1.3,30644,2018,Automatic,Petrol,21400,Cazoo,https://www.cazoo.co.uk/car-details/5a77e9fb-9...
